# Cryptographic operations in Python
This quite extensive Jupyter notebook demonstrates the most important cryptographic operations in Python. 

## Converting strings and bytes

The code snippet makes a conversion from a regular string to a byte string and back again.

In [2]:
# declare a string
myStr = "hello world"

# convert the string to a byte string
myBytesStr = str.encode(myStr)

# print the byte string and its type. A b should be placed in front of the string
print("A byte string:",type(myBytesStr),myBytesStr)

# convert the byte string to a string
myDecStr = myBytesStr.decode()

# print the string and its type
print("A String:", type(myDecStr),myDecStr)

# check that the original string and the decoded string are the same
assert(myStr == myDecStr)


A byte string: <class 'bytes'> b'hello world'
A String: <class 'str'> hello world


## Generating a random number

In order to generate a random number for cryptographic use, it is not wise to use the standard random generator function (in python that is random from the random class). A safe alternative is to use os.urandom, which is suitable for cryptographic use. 

In [3]:
import os

# the following function generates a random number with length of len bytes
def genrandom(len: int):
    # generate a random number, the type of the number is bytes
    rand = os.urandom(len)
    
    # return the random number.
    return rand

result = genrandom(32)
print("Random number: ",type(result),result)

# you can improve the readability, by converting to a hex string
hexResult = result.hex()
print("Hex string:",type(hexResult),hexResult)


Random number:  <class 'bytes'> b"S\xc8\x04|,\xb6K\xec\x8fZ\xed1\x80\xdeC\xb6\xf7\x9c9\x8ef\x14'jGC\x10K\xa0c\xa6\xbc"
Hex string: <class 'str'> 53c8047c2cb64bec8f5aed3180de43b6f79c398e6614276a4743104ba063a6bc


## Create a file with random data

The following code snippet creates a file with random data. This function is used by other parts of this notebook.

In [4]:
import os

def createrandomfile(filename: str, len: int):
    with open(filename,"wb") as f:
        f.write(os.urandom(len))

## Calculate a SHA 256 hash of a string

In [5]:
from cryptography.hazmat.primitives import hashes

def calcSHA256Hash(text: str):
    # convert the string to bytes
    bytesStr = str.encode(text)
    
    # initialize the hashing function
    digest = hashes.Hash(hashes.SHA256())
    
    # add data to the hashing function
    digest.update(bytesStr) # Bytes
    
    # finalize the hashing function and obtain the hash (digestBytes)
    digestBytes = digest.finalize() # Bytes
    
    # convert the hash (bytes string) to a hex string
    hexBytes = digestBytes.hex()
    
    # return the result
    return hexBytes

# What is the hash of nothing? 
print("If you see this specific hash: ", calcSHA256Hash(""), " it is that of nothing, typically an empty file")


If you see this specific hash:  e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855  it is that of nothing, typically an empty file


## Calculate the SHA 256 hash of a file

The following program creates a file, and then reads to calculate a SHA 256 hash. Most important aspects are that the file is read in binary mode (the SHA 256 digest function requires bytes as input) and the the file is read in chunks (pieces of a defined length). Each chunk, read form the file, is added to the digest function by making a call to update. 

In [6]:
## Calculate the SHA 256 hash of a file
from cryptography.hazmat.primitives import hashes

def hashfile(filename: str, chunksize: int):
    # initialize hash function
    digest = hashes.Hash(hashes.SHA256())
    
    # open the file and add data to the hashing function
    with open(filename,"rb") as f:
        # read a chunk of size chunksize from the file
        b = f.read(chunksize)
        
        # Continue until there is nothing more to read
        while b:
            # add the value of b to the digest
            digest.update(b)
            
            # read the next chunk from file
            b = f.read(chunksize)
            
    # finalize the digest to obtain the hash
    digestBytes = digest.finalize()
    
    # convert the bytes string to a hex string
    hexBytes = digestBytes.hex()
    
    # result the result
    return hexBytes

            
# creata random file of length 1000    
createrandomfile("testrandom.dat",1000)

# calculate the hash of this file
result = hashfile("testrandom.dat",1)

# Print the hash. Each time the hash is different as the file is recreated each time
print("The hash is: ", result)


The hash is:  9c2bc08330d94e6b10dbc64d1fb86b989435ee4f7ad815c8239d0faa58164065


## AES encryption and decryption in GCM mode

The following example code performs encryption en decoding using AES in Galois Counter Mode (GCM). This sample is intended for relatively small files. 

In [7]:
import os

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# encrypt a file in finput and write the encrypted file to foutput
def encryptAESGCM(key: bytes, aad: bytes, finput: str, foutput: str):
    # Generate a random initialisation vector (IV)
    iv = os.urandom(16)

    # Construct an AES-GCM Cipher object with the key and the IV
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv)
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    # This is used to verify the integrity of the encrypted data. 
    # Feed aad into the encryptor
    # This is a specific feature of GCM 
    encryptor.authenticate_additional_data(aad)

    # open the output file, where the encrypted data is written
    # note that the file is opened in binary mode
    with open(foutput,"wb") as fo:
        
        # open the input file from where the plaintext data is read
        # note that the file is opened in binary mode
        with open(finput,"rb") as fi:
            
            # read the entire input
            bi = fi.read()
        
            # feed the byte into the encryption function
            ciphertext = encryptor.update(bi) + encryptor.finalize()
            
            # write ciphertext to output
            fo.write(ciphertext)
        
    # return the initialization vector and return the 
    # authentication tag, which is used for integrity verification
    return (iv, encryptor.tag)

# decrypt the file from finput into foutput
def decryptAESGCM(key: bytes, iv: bytes, aad: bytes, tag: bytes, finput: str, foutput: str):
    # Construct a Cipher object, with the key, iv, and additionally the
    # tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(aad)

    # open the output file 
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            
            # read the entire input
            bi = fi.read()
            
            # decrypt the input
            plaintext = decryptor.update(bi) + decryptor.finalize()
            
            # write to output
            fo.write(plaintext)
        
filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# creata random file of length 1000    
createrandomfile(filenameInput,1000)

# create an encryption key
key = os.urandom(16)

# generate a random string that we use for integrity verification
aad = os.urandom(256)

# encrypt filenameInput to filenameEncrypted
iv, tag = encryptAESGCM(key,aad,filenameInput,filenameEncrypted)

# decrypt filenameEncrypted into filennameDecoded
decryptAESGCM(key,iv,aad, tag,filenameEncrypted,filenameDecoded)

# Dump the hashes of the files
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is '33893d3d6ff4c3a8225ef1c718dac44bdda230628979641b62161f7f0a7f63b5'
The SHA526 of 'randomdataEncrypted.dat' is '0de77d40a339b27433cb24a8d609140185f4c0346b91283dda6b7df2dc242553'
The SHA526 of 'randomdataDecoded.dat' is '33893d3d6ff4c3a8225ef1c718dac44bdda230628979641b62161f7f0a7f63b5'
The hashes of the plaintext and decoded file should be the same


## AES encryption and decryption in GCM mode for large files

The following code performs encryption using AES in Galois Counter Mode (GCM). We declare two functions, one for encryption and one for decryption. 

Both functions are implemented to read/write from a file instead of just encrypting/decrypting a single message. This makes it possible to read large files in a stream-like fashion instead of loading the entire file in memory. 

GCM mode does not need padding, so that is not included.


In [10]:
import os

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# encrypt a file in finput and write the encrypted file to foutput
def encryptAESGCM(key: bytes, aad: bytes, finput: str, foutput: str, chunksize: int):
    # Generate a random initialisation vector (IV)
    iv = os.urandom(16)

    # Construct an AES-GCM Cipher object with the key and the IV
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv)
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    # This is used to verify the integrity of the encrypted data. 
    # Feed aad into the encryptor
    # This is a specific feature of GCM 
    encryptor.authenticate_additional_data(aad)

    # open the output file, where the encrypted data is written
    # note that the file is opened in binary mode
    with open(foutput,"wb") as fo:
        
        # open the input file from where the plaintext data is read
        # note that the file is opened in binary mode
        with open(finput,"rb") as fi:
            
            # read chunksize bytes from the input file.
            bi = fi.read(chunksize)
        
            # while bi is not None, there is data to encrypt
            while bi:
                # feed the byte into the encryption function
                # the encryption function may or may not return data
                ciphertext = encryptor.update(bi)
        
                # if there is data, write it to the output file
                if ciphertext:
                    fo.write(ciphertext)
                
                # read chunksize bytes
                bi = fi.read(chunksize)

        # all data is read and the input file is closed
        
        # make the final call to finalize
        ciphertext = encryptor.finalize()
        
        # if there is ciphertext, write that to the file as wll
        if ciphertext:
            fo.write(ciphertext)
        
    # return the initialization vector and return the 
    # authentication tag, which is used for integrity verification
    return (iv, encryptor.tag)

# decrypt the file from finput into foutput
def decryptAESGCM(key: bytes, iv: bytes, aad: bytes, tag: bytes, finput: str, foutput: str, chunksize: int):
    # Construct a Cipher object, with the key, iv, and additionally the
    # tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(aad)

    # open the output file 
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            
            # read chunksize bytes form the input
            bi = fi.read(chunksize)
            
            # while there is a byte read from the input
            while bi:
                
                # decrypt the byte
                plaintext = decryptor.update(bi)
            
                # if the decryptor returned data, write that to the output file
                if plaintext:
                    fo.write(plaintext)
    
                # read the next chunksize bytes from the input
                bi = fi.read(1)
        
        # at this point, all data from the input is read
        # call the finalize function and write the result to the output            
        plaintext = decryptor.finalize()
        if plaintext:
            fo.write(plaintext)
     

filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# creata random file of length 1000 (you may increase this to a large size)    
createrandomfile(filenameInput,1000)

# create an encryption key
key = os.urandom(16)

# generate a random string that we use for integrity verification
aad = os.urandom(256)

# encrypt filenameInput to filenameEncrypted
iv, tag = encryptAESGCM(key,aad,filenameInput,filenameEncrypted,10)

# decrypt filenameEncrypted into filennameDecoded
decryptAESGCM(key,iv,aad, tag,filenameEncrypted,filenameDecoded,10)

# Dump the hashes of the files
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is '355096189cc89663a9b9fe118e6085fc93209ad7a32bcda6a78b78deb8d9e652'
The SHA526 of 'randomdataEncrypted.dat' is '03580dff62ba7d15920e76e71d4c6d260a2b6f6fcfc0ad23e212ea03f6ef4448'
The SHA526 of 'randomdataDecoded.dat' is '355096189cc89663a9b9fe118e6085fc93209ad7a32bcda6a78b78deb8d9e652'
The hashes of the plaintext and decoded file should be the same


## AES Encryption and Decryption in CBC mode

The following script implements two functions for encrypting and decrypting data form a file in AES/CBC mode.  In this example to file is read in one pass.


In [13]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


def encryptAESCBC(key,finput,foutput):
    # generate a new initialisation vector. This is done here to make sure it is renewed, each encryption pass
    iv = os.urandom(16)
    
    # create an encryptor object for the correct algorithm and mode
    encryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).encryptor()

    # open the output file
    with open(foutput,"wb") as fo:
        # open the input file
        with open(finput,"rb") as fi:
            # Create padder, and padding to the input so that the length is correct
            padder = padding.PKCS7(128).padder()
            
            # read the contents of the input
            plaintext = fi.read()
            
            # pad the data
            padded = padder.update(plaintext) + padder.finalize()            
                
            # encrypt the padded data
            ciphertext = encryptor.update(padded) + encryptor.finalize()
            
            # write the ciphertext
            fo.write(ciphertext)
            
    # return the IV, that is needed for decryption
    return iv


def decryptAESCBC(key,iv,finput,foutput):
    # initialize the algorithem and create a decryptor object
    decryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).decryptor()
    
    # open the output file
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            # initialize unpadder
            unpadder = padding.PKCS7(128).unpadder()
            
            # read the ciphertext
            ciphertext = fi.read()
            
            # decrypt data
            decrypted = decryptor.update(ciphertext) + decryptor.finalize() 
                
            # unpad the decrypted data
            unpadded = unpadder.update(decrypted) + unpadder.finalize()

            # write to output
            fo.write(unpadded)
                            
# This is our encryption key
key = bytes.fromhex("41b162773c263f60d4be893792b0c0119a3c1e3d16f881835e879487ca77827d")

# these are the filenames we're using
filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# create a random file of length 1000 bytes
createrandomfile(filenameInput,1000)

# encrypt the file and get the IV
iv = encryptAESCBC(key,filenameInput,filenameEncrypted)

# decrypt the file into a new file
decryptAESCBC(key,iv,filenameEncrypted,filenameDecoded)

# Dump the hashes of the files to check if everything went ok
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is '77a0d43c865b44fd821209d105e36f5940672598cf8c94debd76a30366b15868'
The SHA526 of 'randomdataEncrypted.dat' is '2655742bd71dc748ac47f4ac4c8f2925b07b78a66a4735ef16b3ff44541b36ab'
The SHA526 of 'randomdataDecoded.dat' is '77a0d43c865b44fd821209d105e36f5940672598cf8c94debd76a30366b15868'
The hashes of the plaintext and decoded file should be the same


## AES Encryption and Decryption in CBC mode, large files

The following script implements two functions for encrypting and decrypting data form a file in AES/CBC mode. In this example the file is processed in seperate chunks (a limited number of bytes). This makes it possible to process the file without having to load the entire file into memory.  


In [14]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


def encryptAESCBC(key: bytes,finput: str,foutput: str, chunksize: int):
    # generate a new initialisation vector. This is done here to make sure it is renewed, each encryption pass
    iv = os.urandom(16)
    
    # create an encryptor object for the correct algorithm and mode
    encryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).encryptor()

    # open the output file
    with open(foutput,"wb") as fo:
        # open the input file
        with open(finput,"rb") as fi:
            # Create padder, and padding to the input so that the length is correct
            padder = padding.PKCS7(128).padder()
            
            # read chunksize butes from the input
            plaintext = fi.read(chunksize)
            while plaintext:
                # pad the data
                padded = padder.update(plaintext)            

                # encrypt the data
                ciphertext = encryptor.update(padded)
                
                # write the encrypted data
                fo.write(ciphertext)
                
                # read the next chunk
                plaintext = fi.read(chunksize)
            
            # finalize the padder
            padded = padder.finalize()

            # finalize the encryption
            ciphertext = encryptor.update(padded) + encryptor.finalize()        

            # write the last chunk of ciphertext to the output file
            fo.write(ciphertext)
            
    # return the IV, that is needed for decryption
    return iv


def decryptAESCBC(key: bytes,iv: bytes,finput: str,foutput: str, chunksize: int):
    # initialize the algorithem and create a decryptor object
    decryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).decryptor()
    
    # open the output file
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            # initialize unpadder
            unpadder = padding.PKCS7(128).unpadder()
            
            # read chunksize bytes from the ciphertex
            ciphertext = fi.read(chunksize)

            while ciphertext:
                # decrypt data
                decrypted = decryptor.update(ciphertext) 
                
                # send through unpadder
                unpadded = unpadder.update(decrypted)

                # write to output
                fo.write(unpadded)
                
                # read the next chunksize bytes
                ciphertext = fi.read(chunksize)
        
            # Finalize decryptor 
            decrypted = decryptor.finalize()
    
            # Finalize padder            
            unpadded = unpadder.update(decrypted)
            unpadded += unpadder.finalize()

            # write to the output
            fo.write(unpadded)

# This is our encryption key
key = bytes.fromhex("41b162773c263f60d4be893792b0c0119a3c1e3d16f881835e879487ca77827d")

# these are the filenames we're using
filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# create a random file of length 1000 bytes
createrandomfile(filenameInput,1000)

# encrypt the file and get the IV
iv = encryptAESCBC(key,filenameInput,filenameEncrypted,10)

# decrypt the file into a new file
decryptAESCBC(key,iv,filenameEncrypted,filenameDecoded,10)

# Dump the hashes of the files to check if everything went ok
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is 'b3f231833dd7e73b57aa3025fa835fbc942ca2dff9be949fa6ddb0ea09d3dfea'
The SHA526 of 'randomdataEncrypted.dat' is 'cf0631b00519653f59b556811e9c103e72ce3af66391b67aeb21da7e97059488'
The SHA526 of 'randomdataDecoded.dat' is 'b3f231833dd7e73b57aa3025fa835fbc942ca2dff9be949fa6ddb0ea09d3dfea'
The hashes of the plaintext and decoded file should be the same


## Generating an RSA key pair

In [15]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization


def generate_private_key():
    # generate a private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048, 
        backend=default_backend() 
    )
    return private_key

def store_private_key(private_key,filename: str, password: bytes):
    # convert the private key to pem format
    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        #encryption_algorithm=serialization.NoEncryption()
        encryption_algorithm=serialization.BestAvailableEncryption(password)
    )

    # write the private key to file
    with open(filename,"wb") as f:
        f.write(private_key_pem)
    
def load_private_key(filename: str, password: bytes):
    # read the private key from file
    with open(filename,"rb") as f:
        # read the contents
        data = f.read()
        prvkey = serialization.load_pem_private_key(
            data,
            #password=None
            password=password
        )
        return prvkey
    
def store_public_key(public_key, filename: str):
    # convert the public key to pem
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # write the public key to file
    with open(pubkeyfile,"wb") as f:
        f.write(public_key_pem)

def load_public_key(filename: str):
    # read the public key from file
    with open(filename,"rb") as f:
        # read the contents of the file
        data = f.read()
        pubkey = serialization.load_pem_public_key(data)
    return pubkey

# define filename of the public key
pubkeyfile = "key.pub"
prvkeyfile = "key.prv"
password = b'correcthorsebatterystaple'
    
# generate a private key    
private_key = generate_private_key()

# store the private key
store_private_key(private_key,prvkeyfile,password)

# load the private key from file
private_key = load_private_key(prvkeyfile,password)

# derive public key from private key
public_key = private_key.public_key()

# store the public key to file
store_public_key(public_key,pubkeyfile)

# load the public_key from file
public_key = load_public_key(pubkeyfile)


## Encrypting and decrypting a message with RSA

In [16]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def encryptRSA(pubkey, plaintext: bytes):
    ciphertext = pubkey.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decryptRSA(prvkey, ciphertext: bytes):
    plaintext = prvkey.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext

# message 
message = b'This some random message'

# load public key from file
public_key = load_public_key(pubkeyfile)

# encrypt the message
ciphertext = encryptRSA(public_key,message)

# dump the encrypted message
print("Encrypted message: ", ciphertext.hex())

# load the private key
private_key = load_private_key(prvkeyfile,password)

# decrypt the message
decoded = decryptRSA(private_key, ciphertext)

# dump the message
print("Decoded message: " + decoded.decode())



Encrypted message:  5aa888b7d35bfead5470419c1cdebc3d19620960686b207c4d615bbdee2eb13c27bf1ae88c406e81c4d3c533b718d14c5d7df811352c9e63857240a830de63ba161cfae5879ccf786a09a4bc41d242cf519019d5ed9b9b70de62be293d4059edcdd3a1b4dcaa93f4fb46bf4cd0737fa2ba9dfc7cf8feadfb5acceaa6aa755f77e6bd2a62674cde2e68295580c6a1f63702089da77108b4e91b3c0c4d5850678c3a29848199ee2d314e44a34a11904e0cdd36d7fee030453df1d3d1009c594dd0517550fbc2b3b698b9f89a23958f9f0fbf5842c5b1043a93fafef23d78cb4396da369ab1760379005930e6601fd7973eb326e1e7cf40d8db6825915aa76f6455
Decoded message: This some random message


## Signing and veryfying with RSA


In [17]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature

def sign_message(private_key,message: bytes):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),hashes.SHA256()
    )
    return signature

def verify_message(public_key, message: str, signature: bytes):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False
    

        # define a message
message = b"This is the message to be signed"

# load a private key
private_key = load_private_key(prvkeyfile,password)

# create a signature
signature = sign_message(private_key,message)

# load a public key
public_key = load_public_key(pubkeyfile)

# verify the signature of the message
result = verify_message(public_key, message, signature)

# print the result
print("Verification result (True is expected): ", result)

# verify the signature of another message
result = verify_message(public_key, b"another message", signature)

# print the result
print("Verification result (False is expected): ", result)




Verification result (True is expected):  True
Verification result (False is expected):  False
